In [1]:
import argparse
import icelakes
from icelakes.utilities import encedc, decedc
from icelakes.nsidc import download_granule, edc
from icelakes.detection import read_atl03, detect_lakes, melt_lake

# for the notebook stuff
from IPython.display import Image, display
import pickle
import numpy as np
import traceback

In [2]:
parser = argparse.ArgumentParser(description='Test script to print some stats for a given ICESat-2 ATL03 granule.')
parser.add_argument('--granule', type=str, default='ATL03_20210715182907_03381203_005_01.h5',
                    help='The producer_id of the input ATL03 granule')
parser.add_argument('--polygon', type=str, default='geojsons/jakobshavn_small.geojson',
                    help='The file path of a geojson file for spatial subsetting')
parser.add_argument('--IS2datadir', type=str, default='IS2data',
                    help='The directory into which to download ICESat-2 granules')
parser.add_argument('--download_gtxs', type=str, default='all',
                    help='String value or list of gtx names to download, also accepts "all"')

# set arguments as class for now, to run in jupyter
if parser.prog == 'ipykernel_launcher.py':
    class Args:
        granule = 'ATL03_20210715182907_03381203_005_01.h5'
        polygon = 'geojsons/jakobshavn_test.geojson'
        IS2datadir =  'IS2data'
        download_gtxs = 'all'
    args=Args()
else:
    args = parser.parse_args()

In [3]:
poly_granule_list = [
#     ('geojsons/nivlisen_test.geojson', 'ATL03_20200116042313_03120610_005_01.h5'),
#     ('geojsons/nivlisen_test.geojson', 'ATL03_20200116171056_03200612_005_01.h5'),
#     ('geojsons/georgeVI_test.geojson', 'ATL03_20200204085006_06050610_005_01.h5'),
#     ('geojsons/georgeVI_test.geojson', 'ATL03_20191229231828_00490612_005_01.h5'),
#     ('geojsons/georgeVI_test.geojson', 'ATL03_20191229103045_00410610_005_01.h5'),
#     ('geojsons/georgeVI_test.geojson', 'ATL03_20200131214610_05520612_005_01.h5'),
#     ('geojsons/georgeVI_test.geojson', 'ATL03_20200131085827_05440610_005_01.h5'),
#     ('geojsons/georgeVI_test.geojson', 'ATL03_20200127090648_04830610_005_01.h5'),
#     ('geojsons/georgeVI_test.geojson', 'ATL03_20200127215431_04910612_005_01.h5'),
#     ('geojsons/georgeVI_test.geojson', 'ATL03_20200102102224_01020610_005_01.h5'),
#     ('geojsons/georgeVI_test.geojson', 'ATL03_20200102231007_01100612_005_01.h5'),
#     ('geojsons/georgeVI_test.geojson', 'ATL03_20220224205325_09861410_005_01.h5'),
#     ('geojsons/georgeVI_test.geojson', 'ATL03_20200204213749_06130612_005_01.h5'),
#     ('geojsons/amery_test.geojson', 'ATL03_20190102184312_00810210_005_01.h5'),
#     ('geojsons/amery_test.geojson', 'ATL03_20190103073055_00890212_005_01.h5'),
#     ('geojsons/amery_test.geojson', 'ATL03_20181229185130_00200210_005_01.h5'),
#     ('geojsons/amery_test.geojson', 'ATL03_20190107072235_01500212_005_01.h5'),
#     ('geojsons/amery_test.geojson', 'ATL03_20190119174407_03400210_005_01.h5'),
#     ('geojsons/amery_test.geojson', 'ATL03_20190123173555_04010210_005_01.h5'),
#     ('geojsons/amery_test.geojson', 'ATL03_20190124062339_04090212_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20210715182907_03381203_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20210809171331_07191203_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20210802061504_06051205_005_01.h5'),
#     ('geojsons/79N_test.geojson', 'ATL03_20190822144852_08480405_005_01.h5'),
#     ('geojsons/79N_test.geojson', 'ATL03_20210831140621_10531203_005_01.h5'),
#     ('geojsons/shackleton_test.geojson', 'ATL03_20200224204246_09180610_005_01.h5'),
#     ('geojsons/GrIS-NE-test.geojson', 'ATL03_20210822035912_09091205_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20190609194422_11080305_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20190617064249_12220303_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20190621063429_12830303_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20190716051841_02770403_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20190716180335_02850405_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20190802170446_05440405_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20190810040312_06580403_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20190810164806_06660405_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20190814035453_07190403_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20190814163946_07270405_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20200618131341_12830703_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20200702010756_01020805_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20200706005932_01630805_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20190818163128_07880405_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20190818034635_07800403_005_01.h5'),
#     ('geojsons/jakobshavn_test.geojson', 'ATL03_20220109135030_02791410_005_01.h5'),
#     ('geojsons/amery_test.geojson', 'ATL03_20220105135846_02181410_005_01.h5'),
#     ('geojsons/amery_test.geojson', 'ATL03_20220110023813_02871412_005_01.h5'),
#     ('geojsons/amery_test.geojson', 'ATL03_20220113134210_03401410_005_01.h5'),
#     ('geojsons/amery_test.geojson', 'ATL03_20220118022134_04091412_005_01.h5'),
    ('geojsons/amery_test.geojson', 'ATL03_20220207122631_07211410_005_01.h5'),
    ('geojsons/amery_test.geojson', 'ATL03_20220208011414_07291412_005_01.h5'),
    ('geojsons/amery_test.geojson', 'ATL03_20220220004917_09121412_005_01.h5'),
    ('geojsons/amery_test.geojson', 'ATL03_20220216005735_08511412_005_01.h5'), 
]

In [4]:
for i,(p,g) in enumerate(poly_granule_list):
    
    print(' ')
    print('#################################################################################################################')
    print('### Running script %2i/%2i: combination = (%s, %s)' % (i+1, len(poly_granule_list), g, p))
    print('#################################################################################################################')
    
    args.granule = g
    args.polygon = p
    
    try: 
        
        input_filename = download_granule(args.granule, args.download_gtxs, args.polygon, args.IS2datadir, decedc(edc().u), decedc(edc().p))

        photon_data, bckgrd_data, ancillary = read_atl03(input_filename, geoid_h=True)
        gtx_list = list(photon_data.keys())

        lake_list = []
        for gtx in gtx_list:
            lake_list += detect_lakes(photon_data, gtx, ancillary, args.polygon, verbose=False)

        # from here on it's for plotting and saving data
        quality_list = []
        for i,lake in enumerate(lake_list):
            quality_list.append(lake.detection_quality)

        sort_idxs = np.argsort(1-np.array(quality_list))
        lake_list_ordered = [lake_list[i] for i in sort_idxs]

        for lake in lake_list_ordered:

            fig = lake.plot_detected(min_width=0.0, min_depth=0.0)
            filename_base = 'lake_%05i_%s_%s_%s_%s_%s.jpg' % ((1-lake.detection_quality)*10000, lake.ice_sheet, lake.melt_season, 
                                                                  lake.polygon_name, lake.granule_id[:-4], lake.gtx)
            figname = 'figs_quality/%s.jpg' % filename_base
            if fig is not None: fig.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0)
            if lake.detection_quality>0.1: Image(figname)

            pklname = 'pickles/%s.pkl' % filename_base
            with open(pklname, 'wb') as f: pickle.dump(vars(lake), f)
            print('wrote to file:', pklname)
            print('_______________________________________________________________________________', end='\n\n')
            
    except:
        traceback.print_exc()

 
#################################################################################################################
### Running script  1/ 4: combination = (ATL03_20220207122631_07211410_005_01.h5, geojsons/amery_test.geojson)
#################################################################################################################

  ATL03_20220207122631_07211410_005_01.h5, 2861.14 MB

Input geojson: geojsons/amery_test.geojson
Simplified polygon coordinates based on geojson input: 70.7185132151385,-68.96962937607141,68.5212475901385,-68.67593887984046,68.16766563716442,-69.37609423932955,67.14593712153942,-69.96386398841524,67.24481407466442,-70.33689686702512,67.55243126216442,-70.46220216940445,67.3908919895423,-70.86990014673427,66.7976302707923,-71.50750229823859,66.4900130832923,-72.27235679731974,64.6662826145423,-72.76081712773662,62.7766341770423,-72.99370412007747,62.4909896457923,-73.33101750118587,63.3259505832923,-73.77271760375082,66.9294662082923,-74.220954850208